### Reference: Deep Learning by Ian Goodfellow, Yoshua Bengio and Aaron Courville

In a neural network, a parametric model defines a distribution $p(y | x;\theta)$ and the principle of maximum likelihood is applied. That means we use cross-entropy between the training data and the model's predictions as the cost function.

__In information theory, the cross-entropy between two probability distributions $p$ and $q$ over the same underlying set of events measures the average number of bits needed to identify an event drawn from the set if a coding scheme used for the set is optimized for an estimated probability distribution $q$, rather than the true distribution $p$.__

## Cost Functions

#### Learning Conditional Distributions with Maximum Likelihood

Most modern neural networks are trained using maximum likelihood. This means that the cost function is simply the negative log-likelihood, equivalently described as the cross-entropy between the training data and the model distribution. This cost function is given by,

$$
J(\theta) = - E_{x,y~\hat{p}_{data}} log P_{model}(y|x)
$$

The specific form of the cost function changes from model to model depending on the specific form of the $log P_{model}$.

#### Learning Conditional Statistics

Instead of learning a full probability distribution $p(y|x;\theta)$ we often want to learn just one conditional statistic of $y$ given $x$. For example, we may have a predictor $f(x;\theta)$ that we want to use to predict the mean of $y$.

From this perspective, we can view the cost function as being a __functional__ rather than just a function. A functional is a mapping from functions to real numbers. We can thus think of learning as choosing a function rather than merely choosing a set of parameters. 

A result derived using __Calculus of Variations__ is necessary for this, solving the optimization problem,

$$
f^{*} = argmin_{f} E{x, y ~ p_{data}} \bigl\| y - f(x) \bigl\|^{2}
$$

yields the result,

$$
f^{*}(x) = E_{y~p_{data(y|x)}}[y]
$$

as long as this function lies within the class we optimize over.

## Output Units

#### Linear Units for Gaussian Output Distributions

Given features $h$, a layer of linear output units produces a vector $\hat{y} = W^{T}h + b$.

Linear output layers are often used to produce the mean of a conditional Gaussian distribution,

$$
p(y|x) = \mathcal{N}(y; \hat{y}, I)
$$

#### Sigmoid Units for Bernoulli Output Distributions

Many tasks require predicting the value of a binary variable $y$. The maximum likelihood approach is to define a Bernoulli distribution over $y$ conditioned on $x$.

A Bernoulli distribution is defined by just a single number. The neural net needs to predict only $P(y=1|x)$. For this number to be a valid probability, $x$ must lie within the unit interval $[0,1]$. 

This approach has a significant limitation that does not allow us to train it using gradient descent very effectively. Suppose we were to use a linear unit, and threshold its value to obtain a valid probability:

$$
P(y=1|x) = max\bigl\{0, min\bigl\{1, w^{T}h + b\bigl\}\bigl\}
$$

Anytime $w^{T}h + b$ will stray outside the unit interval, the gradient of the output of the model with respect to its parameters would be $0$. That is problematic because then our learning algorithm no longer has a guide for how to improve the corresponding parameters. For example,

$$
P(y=1|x) = max\bigl\{0, min\bigl\{1, -1.3\bigl\}\bigl\} \\[0.2cm]
\implies P(y=1|x) = max\bigl\{0, -1.3\bigl\} \\[0.2cm]
\implies P(y=1|x) = 0
$$

This limitation is overcome by using Sigmoid output units defined by,

$$
\hat{y} = \sigma(w^{T}h + b)
$$

We can think of the sigmoid output unit as having two components. First it uses a linear layer to compute $z = w^{T}h + b$. Next, it uses sigmoid activation function to convert $z$ into a probability. 

We can derive the Bernoulli distribution controlled by a sigmoidal transformation z as follows:

$$
log\tilde{P(y)} = yz \\[0.2cm]
\implies \tilde{P(y)} = e^{yz} \\[0.2cm]
\implies P(y) = \frac{e^{yz}}{\sum_{y'=0}^{1} e^{y'z}} \\[0.2cm]
\implies P(y) = \sigma((2y-1)z)
$$

The loss function for maximum likelihood learning of a Bernoulli parametrized by a sigmoid is,

$$
\begin{align*}
J(\theta) = -logP(y|x) \\[0.2cm]
= -log \sigma((2y-1)z) \\[0.2cm]
\therefore J(\theta) = \zeta ((1-2y)z)
\end{align*}
$$

__Mathematical Prerequisites__

__The logistic Sigmoid:__
$$
\sigma(x) = \frac{1}{1+ e^{-x}}
$$

__The Softplus Function:__
$$
\zeta(x) = log(1 + e^{x})
$$

The softplus function commonly arises when manipulating expressions involving sigmoids. A useful property of the softplus function that we have applied in the mathematical derivation above is,
$$
log \sigma(x) = -\zeta(-x)
$$

#### Softmax Units for Multinoulli Output Distributions

A softmax function is used when we want to represent a probability distribution over a discrete variable with n possible values. This can be seen as a generilzation of the sigmoid function which was used to represent a probability distribution over a binary variable. 